In [ ]:
# Step 1: Install required packages
!pip install feedparser beautifulsoup4 openai

In [2]:
# Step 2: Download and parse the RSS feed
import feedparser
def get_feed(url):
    return feedparser.parse(url)

In [3]:
# Step 3: Extract HTML from the description
from bs4 import BeautifulSoup

def extract_text(html):
    soup = BeautifulSoup(html, "html.parser")
    return soup.get_text()

In [23]:
import openai
openai.api_key = "insert-your-API-key-here"


# Step 4: Summarize using GPT-4
def summarize_text(text):
    response = openai.Completion.create(
        engine="text-davinci-002", 
        prompt=f"Please provide a short summary of the following text and remove unnecesary swhitespace: {text}",
        temperature=0.7,
        max_tokens=50,
        top_p=1,
        frequency_penalty=0.5,
        presence_penalty=0,
    )
    return response.choices[0].text.strip()

# Step 5: Sentiment classification
def sentiment_classification(text):
    response = openai.Completion.create(
        engine="text-davinci-002", 
        prompt=f"Please classify this breaking financial news article on a range of -1 to 1 with 0.1 granularity, where -1 is maximum bearishness and 1 is maximum bullishness: {text}",
        temperature=0.7,
        max_tokens=50,
        top_p=1,
        frequency_penalty=0.5,
        presence_penalty=0,
    )
    return response.choices[0].text.strip()

In [5]:
# Step 6: Store the summarized information in a SQLite database
import sqlite3

def create_db():
    conn = sqlite3.connect(":memory:")
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE news (title TEXT, summary TEXT, sentiment TEXT)''')
    return conn, cursor

def insert_news(cursor, title, summary, sentiment=None):
    cursor.execute("INSERT INTO news (title, summary, sentiment) VALUES (?, ?, ?)", (title, summary, sentiment))


In [24]:
# Step 7: Continuously monitor the RSS feed for updates
import time

def monitor_feed(url, conn, cursor, interval=60):
    seen_titles = set()
    while True:
        feed = get_feed(url)
        for entry in feed.entries:
            title = entry.title
            if title not in seen_titles:
                seen_titles.add(title)
                description = extract_text(entry.description)
                summary = summarize_text(description[:4000])
                sentiment = sentiment_classification(description[:4000])  
                insert_news(cursor, title, summary, sentiment)  
                print(f"Inserted news: {title} : {summary} : {sentiment}")
        time.sleep(interval)

In [25]:
'''
# This code will download the RSS feed and process new entries every 60 seconds. 
Adjust the interval parameter in monitor_feed() to control how often the feed is checked.
'''
if __name__ == "__main__":
    url = "https://www.forexlive.com/feed"
    conn, cursor = create_db()
    monitor_feed(url, conn, cursor)

Inserted news: It's been quiet this year in EUR/USD trading... too quiet : As we wrap up the first quarter of the year, Deutsche Bank has a look at the euro and notes that EUR/USD volatility and the range of the pair has been surprisingly narrow, particularly given the volatility in fixed income.
One of : 0.5
Inserted news: USDCAD approaches key 100 day moving average : The USDCAD has been down for four consecutive days and is approaching its 100 day moving average. A move below that level would increase the bearish bias. : 0
Inserted news: US weekly EIA natural gas inventories -47B vs -54B expected : May Henry Hub natural gas fell below $2.00 yesterday but is up to $2.13 today as we get through the roll. This report won't help. : -0.4
Inserted news: The GBPUSD moves away from it's 100 hour MA. More bullish bias for the pair. : The GBPUSD is pushing higher in trading today and in the process has seen the price move away from its 100 year moving average (blue line in the chart above). T

KeyboardInterrupt: ignored